# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 2 2022 2:54PM,245664,16,SHL-8476270,"SHL Pharma, LLC",Released
1,Sep 2 2022 2:52PM,245663,10,EYE082922EM.1,Eye Pharma Inc,Released
2,Sep 2 2022 2:36PM,245662,10,SO94371,"Senseonics, Incorporated",Released
3,Sep 2 2022 2:36PM,245662,10,SO94372,"Senseonics, Incorporated",Released
4,Sep 2 2022 2:36PM,245662,10,SO94373,"Senseonics, Incorporated",Released
5,Sep 2 2022 2:36PM,245662,10,SO94374,"Senseonics, Incorporated",Released
6,Sep 2 2022 2:36PM,245661,10,0085959399,ISDIN Corporation,Released
7,Sep 2 2022 2:36PM,245661,10,0085959398,ISDIN Corporation,Released
8,Sep 2 2022 2:36PM,245661,10,0085959400,ISDIN Corporation,Released
9,Sep 2 2022 2:36PM,245661,10,0085959407,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,245660,Released,2
36,245661,Released,6
37,245662,Released,4
38,245663,Released,1
39,245664,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
245660,NaN,NaN,2.0
245661,NaN,NaN,6.0
245662,NaN,NaN,4.0
245663,NaN,NaN,1.0
245664,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245562,9.0,4.0,1.0
245563,0.0,0.0,1.0
245564,0.0,0.0,1.0
245570,0.0,0.0,1.0
245571,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245562,9,4,1
245563,0,0,1
245564,0,0,1
245570,0,0,1
245571,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245562,9,4,1
1,245563,0,0,1
2,245564,0,0,1
3,245570,0,0,1
4,245571,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245562,9,4,1
1,245563,,,1
2,245564,,,1
3,245570,,,1
4,245571,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 2 2022 2:54PM,245664,16,"SHL Pharma, LLC"
1,Sep 2 2022 2:52PM,245663,10,Eye Pharma Inc
2,Sep 2 2022 2:36PM,245662,10,"Senseonics, Incorporated"
6,Sep 2 2022 2:36PM,245661,10,ISDIN Corporation
12,Sep 2 2022 2:35PM,245655,10,ISDIN Corporation
19,Sep 2 2022 2:34PM,245660,10,Emerginnova
21,Sep 2 2022 2:32PM,245658,20,"Exact-Rx, Inc."
44,Sep 2 2022 2:17PM,245653,10,"Methapharm, Inc."
53,Sep 2 2022 2:16PM,245643,10,ISDIN Corporation
81,Sep 2 2022 1:43PM,245646,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 2 2022 2:54PM,245664,16,"SHL Pharma, LLC",,,1
1,Sep 2 2022 2:52PM,245663,10,Eye Pharma Inc,,,1
2,Sep 2 2022 2:36PM,245662,10,"Senseonics, Incorporated",,,4
3,Sep 2 2022 2:36PM,245661,10,ISDIN Corporation,,,6
4,Sep 2 2022 2:35PM,245655,10,ISDIN Corporation,,,7
5,Sep 2 2022 2:34PM,245660,10,Emerginnova,,,2
6,Sep 2 2022 2:32PM,245658,20,"Exact-Rx, Inc.",,,23
7,Sep 2 2022 2:17PM,245653,10,"Methapharm, Inc.",,,9
8,Sep 2 2022 2:16PM,245643,10,ISDIN Corporation,,,28
9,Sep 2 2022 1:43PM,245646,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 2 2022 2:54PM,245664,16,"SHL Pharma, LLC",1,,
1,Sep 2 2022 2:52PM,245663,10,Eye Pharma Inc,1,,
2,Sep 2 2022 2:36PM,245662,10,"Senseonics, Incorporated",4,,
3,Sep 2 2022 2:36PM,245661,10,ISDIN Corporation,6,,
4,Sep 2 2022 2:35PM,245655,10,ISDIN Corporation,7,,
5,Sep 2 2022 2:34PM,245660,10,Emerginnova,2,,
6,Sep 2 2022 2:32PM,245658,20,"Exact-Rx, Inc.",23,,
7,Sep 2 2022 2:17PM,245653,10,"Methapharm, Inc.",9,,
8,Sep 2 2022 2:16PM,245643,10,ISDIN Corporation,28,,
9,Sep 2 2022 1:43PM,245646,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 2 2022 2:54PM,245664,16,"SHL Pharma, LLC",1,,
1,Sep 2 2022 2:52PM,245663,10,Eye Pharma Inc,1,,
2,Sep 2 2022 2:36PM,245662,10,"Senseonics, Incorporated",4,,
3,Sep 2 2022 2:36PM,245661,10,ISDIN Corporation,6,,
4,Sep 2 2022 2:35PM,245655,10,ISDIN Corporation,7,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 2 2022 2:54PM,245664,16,"SHL Pharma, LLC",1.0,NaN,NaN
1,Sep 2 2022 2:52PM,245663,10,Eye Pharma Inc,1.0,NaN,NaN
2,Sep 2 2022 2:36PM,245662,10,"Senseonics, Incorporated",4.0,NaN,NaN
3,Sep 2 2022 2:36PM,245661,10,ISDIN Corporation,6.0,NaN,NaN
4,Sep 2 2022 2:35PM,245655,10,ISDIN Corporation,7.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 2 2022 2:54PM,245664,16,"SHL Pharma, LLC",1.0,0.0,0.0
1,Sep 2 2022 2:52PM,245663,10,Eye Pharma Inc,1.0,0.0,0.0
2,Sep 2 2022 2:36PM,245662,10,"Senseonics, Incorporated",4.0,0.0,0.0
3,Sep 2 2022 2:36PM,245661,10,ISDIN Corporation,6.0,0.0,0.0
4,Sep 2 2022 2:35PM,245655,10,ISDIN Corporation,7.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2702046,85.0,23.0,5.0
12,491271,4.0,0.0,0.0
15,245595,3.0,0.0,0.0
16,2701698,10.0,1.0,0.0
17,245585,1.0,0.0,0.0
18,245624,1.0,0.0,0.0
20,1473613,42.0,4.0,10.0
21,491290,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2702046,85.0,23.0,5.0
1,12,491271,4.0,0.0,0.0
2,15,245595,3.0,0.0,0.0
3,16,2701698,10.0,1.0,0.0
4,17,245585,1.0,0.0,0.0
5,18,245624,1.0,0.0,0.0
6,20,1473613,42.0,4.0,10.0
7,21,491290,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,85.0,23.0,5.0
1,12,4.0,0.0,0.0
2,15,3.0,0.0,0.0
3,16,10.0,1.0,0.0
4,17,1.0,0.0,0.0
5,18,1.0,0.0,0.0
6,20,42.0,4.0,10.0
7,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,85.0
1,12,Released,4.0
2,15,Released,3.0
3,16,Released,10.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,18,20,21
Status,,,,,,,,
Completed,5.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0
Executing,23.0,0.0,0.0,1.0,0.0,0.0,4.0,1.0
Released,85.0,4.0,3.0,10.0,1.0,1.0,42.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,18,20,21
0,Completed,5.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0
1,Executing,23.0,0.0,0.0,1.0,0.0,0.0,4.0,1.0
2,Released,85.0,4.0,3.0,10.0,1.0,1.0,42.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,18,20,21
0,Completed,5.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0
1,Executing,23.0,0.0,0.0,1.0,0.0,0.0,4.0,1.0
2,Released,85.0,4.0,3.0,10.0,1.0,1.0,42.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()